In [2]:
import certifi
import requests

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import re

# Set up the Chrome WebDriver
chrome_driver_path = "D:/chromedriver-win64/chromedriver.exe"  # Replace with your ChromeDriver path
service = Service(chrome_driver_path)
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")

# Initialize the WebDriver
driver = webdriver.Chrome(service=service, options=options)

def search_tweets_by_geolocation(latitude, longitude, radius, place_name, min_tweets=50):
    # Twitter search by geolocation for latest tweets
    search_url_geo = f"https://twitter.com/search?q=geocode%3A{latitude}%2C{longitude}%2C{radius}&f=live"
    
    # Twitter search mentioning the place for latest tweets
    search_url_place = f"https://twitter.com/search?q={place_name}&f=live"

    # Data storage for tweets
    tweets_data = []

    # Function to collect tweets from the current page
    def collect_tweets():
        new_tweets = driver.find_elements(By.CSS_SELECTOR, "[data-testid='tweetText']")
        for tweet in new_tweets:
            tweet_text = tweet.text
            tweets_data.append({
                "Tweet": tweet_text,
                "Geolocation": f"{latitude}, {longitude}"
            })
            print(tweet_text)

    # Search for the latest tweets posted from the geolocation
    print(f"Searching for the most recent tweets posted from geolocation: {latitude}, {longitude} within {radius}")
    driver.get(search_url_geo)
    time.sleep(5)
    
    # Scroll and collect until at least min_tweets are gathered
    while len(tweets_data) < min_tweets:
        collect_tweets()
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    # Now search for the latest tweets mentioning the place
    print(f"\nSearching for the most recent tweets mentioning: {place_name}")
    driver.get(search_url_place)
    time.sleep(5)
    
    # Scroll and collect until at least min_tweets more tweets are gathered
    initial_count = len(tweets_data)
    while len(tweets_data) < initial_count + min_tweets:
        collect_tweets()
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    # Export the tweets to a CSV file
    df = pd.DataFrame(tweets_data)
    csv_filename = f"latest_tweets_data.csv"
    df.to_csv(csv_filename, index=False)
    print(f"Latest tweets data exported to {csv_filename}")

# Example usage with latitude, longitude, radius, and place_name
latitude = 24.7941004
longitude = 93.1170986
radius = "1000km"  # Search within a 50 km radius
place_name = "Tamil Nadu"  # Place name to search for mentions

# Search and export the results
search_tweets_by_geolocation(latitude, longitude, radius, place_name, min_tweets=50)

# Close the browser
driver.quit()


Searching for the most recent tweets posted from geolocation: 11.059821, 78.387451 within 1000km
తెలంగాణ బీజేపీ శాసనసభ్యులతో, ఎంపీలతో చాలా మంచి సమావేశం జరిగింది.

రాష్ట్రంలో మా పార్టీ ఉనికి వేగంగా విస్తరిస్తోంది. తెలంగాణ ప్రజలు ఇప్పటికే కాంగ్రెస్‌తో విసిగిపోయారు అంతేకాక బీఆర్‌ఎస్ దుష్టపాలన వల్ల కలిగిన భయంకరమైన జ్ఞాపకాలతో ఉన్నారు. ఎంతో ఆశతో బీజేపీ వైపు చూస్తున్నారు.
You are so much more than the media now
Indian culture resonates globally! 

Wherever I go, I see immense enthusiasm towards our history and culture, which is extremely gladdening. Here is a glimpse…
 Happy Thanksgiving! 
37-year-old Lionel Messi is still being nominated for The Best FIFA Men's Player in 2024.

We're not worthy 
The Indian Cricket Team were hosted by the Honourable Anthony Albanese MP, Prime Minister of Australia at the Parliament House, Canberra. 

#TeamIndia will take part in a two-day pink ball match against PM XI starting Saturday.
BIG BREAKING NEWS  PM Modi held high level meeting on Bangladesh crisis 


In [1]:
import pandas as pd
import re
import nltk

# Download NLTK resources if necessary
nltk.download('stopwords')
from nltk.corpus import stopwords

# Load the CSV file
csv_filename = "latest_tweets_data.csv"
tweets_data = pd.read_csv(csv_filename)

# Ensure all entries in the Tweet column are strings (replace NaN with an empty string)
tweets_data['Tweet'] = tweets_data['Tweet'].fillna('').astype(str)

# Function to clean tweet text while keeping punctuations like !, ?, .
def clean_tweet(text):
    try:
        # Remove URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        
        # Remove mentions (@username) and hashtags (#hashtag)
        text = re.sub(r'@\w+|#\w+', '', text)
        
        # Remove non-ASCII characters (non-English characters)
        text = text.encode('ascii', 'ignore').decode('ascii')
        
        # Remove special characters except for meaningful punctuation (!, ?, ., etc.)
        text = re.sub(r"[^\w\s!?.,]", '', text)
        
        # Convert to lowercase
        text = text.lower()
        
        # Remove extra spaces
        text = re.sub(r'\s+', ' ', text).strip()
        
        return text
    except Exception as e:
        print(f"Error processing tweet: {text}, Error: {e}")
        return ''

# Optional: Remove stopwords (if desired)
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

# Apply the cleaning function to the tweets
tweets_data['Cleaned_Tweet'] = tweets_data['Tweet'].apply(clean_tweet)

# Optional: Uncomment the next line to remove stopwords
# tweets_data['Cleaned_Tweet'] = tweets_data['Cleaned_Tweet'].apply(remove_stopwords)

# Save the cleaned tweets back to a new CSV file
cleaned_csv_filename = "cleaned_tweets_data_with_punctuation.csv"
tweets_data.to_csv(cleaned_csv_filename, index=False)

print(f"Cleaned tweets data exported to {cleaned_csv_filename}")


Cleaned tweets data exported to cleaned_tweets_data_with_punctuation.csv


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mehul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
# Display the first few entries of the cleaned data
try:
    cleaned_csv_filename = "cleaned_tweets_data_with_punctuation.csv"
    cleaned_tweets_data = pd.read_csv(cleaned_csv_filename)
    cleaned_tweets_data.head()
    print(tweets_data[['Cleaned_Tweet']].head(10))

except FileNotFoundError:
    print(f"File '{cleaned_csv_filename}' not found. Ensure the cleaning process ran successfully and the file exists.")



                                       Cleaned_Tweet
0  had a very good meeting with legislators and m...
1                 hope you had a great thanksgiving!
2  elon musk, donald trump, barron and melania bl...
3  i bow to the land of mahaprabhu jagannath. the...
4  37yearold lionel messi is still being nominate...
5            you are so much more than the media now
6  the indian cricket team were hosted by the hon...
7  jude bellingham waited by liverpools dressing ...
8                                               dave
9                                        back tattoo


In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from branca.element import Figure

# Load the cleaned tweets data
cleaned_csv_filename = "cleaned_tweets_data_with_punctuation.csv"
tweets_data = pd.read_csv(cleaned_csv_filename)

# Ensure all entries in the Tweet column are strings (replace NaN with an empty string)
tweets_data['Cleaned_Tweet'] = tweets_data['Cleaned_Tweet'].fillna('').astype(str)

# Generate a popularity metric (e.g., length of the cleaned tweet text)
tweets_data['Popularity'] = tweets_data['Cleaned_Tweet'].apply(len)

# Drop tweets with cleaned length shorter than 10 characters
tweets_data = tweets_data[tweets_data['Popularity'] >= 10]

# Parse geolocation into separate Latitude and Longitude columns
tweets_data[['Latitude', 'Longitude']] = tweets_data['Geolocation'].str.split(',', expand=True).astype(float)

# Ensure all filtered entries are being used
print(f"Total tweets after filtering: {len(tweets_data)}")  # Logs the count of tweets being processed

# Calculate the center of the map based on all tweet locations
map_center = [tweets_data['Latitude'].mean(), tweets_data['Longitude'].mean()]

# Initialize the folium map
fig = Figure(width=800, height=600)
tweet_map = folium.Map(location=map_center, zoom_start=6)
fig.add_child(tweet_map)

# Add a marker cluster to the map for the general tweet area
marker_cluster = MarkerCluster().add_to(tweet_map)

# Tweet-styled popup HTML template
def create_tweet_popup(tweet_text):
    return folium.Popup(
        html=f"""
        <div style="font-family: Arial, sans-serif; font-size: 12px; line-height: 1.5; max-width: 300px; padding: 10px; 
                    border: 1px solid #e1e8ed; border-radius: 10px; box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);">
            <div style="display: flex; align-items: center; margin-bottom: 10px;">
                <img src="https://cdn-icons-png.flaticon.com/512/733/733579.png" width="30" height="30" style="margin-right: 10px; border-radius: 50%;">
                <b style="color: #1da1f2;">TwitterTracker</b>
            </div>
            <p style="margin: 0; color: #14171a;">{tweet_text}</p>
            <hr style="border: none; border-top: 1px solid #e1e8ed; margin: 10px 0;">
            <div style="display: flex; justify-content: space-between; font-size: 10px; color: #657786;">
                <span>12:34 PM · Nov 29, 2024</span>
                <span>Twitter Web App</span>
            </div>
        </div>
        """,
        max_width=350,
    )

# Add all tweet locations to the map using styled popups
for _, tweet in tweets_data.iterrows():
    folium.Marker(
        location=[tweet['Latitude'], tweet['Longitude']],
        popup=create_tweet_popup(tweet['Cleaned_Tweet']),
        icon=folium.Icon(color="blue", icon="info-sign"),
    ).add_to(marker_cluster)

# Highlight the top tweets with tweet-styled popups and custom icons
top_tweets = tweets_data.nlargest(10, 'Popularity')
for _, tweet in top_tweets.iterrows():
    folium.Marker(
        location=[tweet['Latitude'], tweet['Longitude']],
        popup=create_tweet_popup(tweet['Cleaned_Tweet']),
        icon=folium.CustomIcon(
            "https://cdn-icons-png.flaticon.com/512/733/733579.png",  # Twitter logo
            icon_size=(30, 30)
        ),
    ).add_to(tweet_map)

# Save the map as an HTML file
tweet_map.save("cleaned_tweets_map.html")

print("Map saved as cleaned_tweets_map.html. Open it in a browser to view.")


Total tweets after filtering: 87
Map saved as cleaned_tweets_map.html. Open it in a browser to view.


In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from multiprocessing import cpu_count, Pool
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import joblib

# Download stopwords and lemmatizer data if not already downloaded
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Compile regular expressions once for optimization
mention_pattern = re.compile(r'@\w+')
hashtag_pattern = re.compile(r'#')
url_pattern = re.compile(r"http\S+|www\S+|https\S+")
emoji_pattern = re.compile(r'[\U00010000-\U0010FFFF]')
special_char_pattern = re.compile(r'[^A-Za-z\s]')

# Define preprocessing function
def preprocess_tweet_optimized(tweet):
    # Convert to lowercase
    tweet = tweet.lower()
    # Remove mentions
    tweet = mention_pattern.sub('', tweet)
    # Remove "#" symbol but retain the text
    tweet = hashtag_pattern.sub('', tweet)
    # Remove URLs
    tweet = url_pattern.sub('', tweet)
    # Remove emojis
    tweet = emoji_pattern.sub('', tweet)
    # Remove special characters, numbers, and punctuation (except words)
    tweet = special_char_pattern.sub('', tweet)
    # Tokenize and remove stopwords
    words = [word for word in tweet.split() if word not in stop_words]
    # Lemmatize words
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

# Load the cleaned tweets data
cleaned_csv_filename = "cleaned_tweets_data_with_punctuation.csv"
tweets_data = pd.read_csv(cleaned_csv_filename)

# Ensure all entries in the Tweet column are strings (replace NaN with an empty string)
tweets_data['Cleaned_Tweet'] = tweets_data['Cleaned_Tweet'].fillna('').astype(str)

# Generate a popularity metric (e.g., length of the cleaned tweet text)
tweets_data['Popularity'] = tweets_data['Cleaned_Tweet'].apply(len)

# Drop tweets with cleaned length shorter than 10 characters
tweets_data = tweets_data[tweets_data['Popularity'] >= 10]

# Parse geolocation into separate Latitude and Longitude columns
tweets_data[['Latitude', 'Longitude']] = tweets_data['Geolocation'].str.split(',', expand=True).astype(float)

# Apply the preprocessing function to each tweet in the 'Cleaned_Tweet' column
tweets_data['cleaned_tweet'] = tweets_data['Cleaned_Tweet'].apply(preprocess_tweet_optimized)


# Load the saved TF-IDF Vectorizer
tfidf_vectorizer = joblib.load('tfidf_vectorizer.joblib')

# Vectorize the cleaned tweets
X = tfidf_vectorizer.fit_transform(tweets_data['cleaned_tweet'])

# Load your pretrained model (this assumes you've saved it in a file)
pretrained_model = joblib.load("final_mlp_model.joblib")  # Make sure to replace with the actual path

# Predict whether each tweet is a crisis or not
y_pred = pretrained_model.predict(X)

# Add predictions to the dataframe
tweets_data['Prediction'] = y_pred

# Filter out non-crisis tweets
crisis_tweets = tweets_data[tweets_data['Prediction'] == 1]  # Assuming '1' indicates a crisis tweet

# Print the filtered crisis tweets
print(crisis_tweets[['Cleaned_Tweet', 'Prediction']].head())


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mehul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mehul\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


ValueError: X has 1915 features, but MLPClassifier is expecting 5000 features as input.

In [1]:
import joblib
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Load a sample dataset
data = load_iris()
X = data.data
y = data.target

# Train a simple Logistic Regression model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

# Save the model using joblib
joblib.dump(model, 'sample_model.pkl')

['sample_model.pkl']